In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import risk_kit as rk

A major lottery advertises that it pays the winner `$10 million`. However this prize money is paid at the rate of `$500,000` each year (with the first payment being immediate) for a total of $20$ payments. What is the present value of this prize at $10%$ interest compounded annually?

$$
\begin{align}
PV &= A\sum_{i=0}^{T-1} \frac{1}{(1+r)^t} \\
&= \frac{A(1+r)}{r}(1-\frac{1}{(1+r)^T}) \\
&= \frac{500,000(1+0.1)}{0.1}(1-\frac{1}{1.01^{20}}) \\
&= 4.6825M
\end{align}
$$

In [2]:
cash_flows = pd.Series([500000] * 20)
discount_rate = 0.1
unit = 1000000
rk.present_value(cash_flows, discount_rate) / unit

0    4.68246
dtype: float64

### Question 2 (Sunk Costs (Exercise 2.6 in Luenberger))

A young couple has made a deposit of the first month's rent (equal to `$1,000`) on a 6-month apartment lease. The deposit is refundable at the end of six months if they stay until the end of the lease.

The next day they find a different apartment that they like just as well, but its monthly rent is only `$900`. And they would again have to put a deposit of `$900` refundable at the end of 6 months.

They plan to be in the apartment only 6 months. Should they switch to the new apartment? Assume an (admittedly unrealistic!) interest rate of $12%$ per month compounded monthly.

Compare the two alternatives: Stay in the original apartment, for an NPV, $C_1$, given by
$$C_1 = -\sum_{i=0}^5 \frac{1000}{1.12^i} + \frac{1000}{1.12^6} \approx -4098$$
Take the new apartment where we assume a security deposit is again required. The NPV then is 
$$C_2 = -900 - \sum_{i=0}^5 \frac{900}{1.12^i} + \frac{900}{1.12^6} \approx -4588$$
The couple should take the $1000 apartment.

In [3]:
# Assuming the desposited amount is not refundable if they chose to switch

cash_flow_stay = pd.Series([0] + [-1000] * 5 + [1000])
cash_flow_switch = pd.Series([-1800] + [-900] * 5 + [900])
discount_rate = 0.12
stay_cost = float(rk.present_value(cash_flow_stay, discount_rate))
switch_cost = float(rk.present_value(cash_flow_switch, discount_rate))
stay_cost, switch_cost

C:\Users\x_tia\AppData\Local\Temp\ipykernel_14744\1653239865.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  stay_cost = float(rk.present_value(cash_flow_stay, discount_rate))
C:\Users\x_tia\AppData\Local\Temp\ipykernel_14744\1653239865.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  switch_cost = float(rk.present_value(cash_flow_switch, discount_rate))


(-3098.1450811676837, -4588.330573050915)

### Question 3 (Relation between spot and discount rates)

Suppose the spot rates for $1$ and $2$ years are $s_1 = 6.3%$ and $s_2 = 6.9%$ with annual compounding. Recall that in this course interest rates are always quoted on an annual basis unless otherwise specified. What is the discount rate $d(0,2)$?

From the relation in the slides we get
$$
d(0,2) = \frac{1}{(1+s_2)^2} = 0.8751
$$

In [4]:
s1 = 0.063
s2 = 0.069

rk.discount(2, 0.069)

0.8750736155679097

### Question 4 (Relation between spot and forward rates)

Suppose the spot rates for $1$ and $2$ years are $s_1 = 6.3%$ and $s_2 = 6.9%$ with annual compounding. Recall that in this course interest rates are always quoted on an annual basis unless otherwise specified. What is the forward rate, $f_{1,2}$ assuming annual compounding?

From the relation in the slides we get 
$$
\begin{align}
(1+s_2)^2 &= (1+s_1)(1+f_{1,2}) \\
\Rightarrow f_{1,2} &= \frac{(1+s_2)^2}{(1+s_1)} - 1 \\
&= 0.075 = 7.5\%
\end{align}
$$

In [5]:
f_12 = rk.discount(1, s1) / rk.discount(2, s2) - 1
f_12 * 100

7.503386641580434

### Question 5 (Forward contract on a stock)

The current price of a stock is `$400` per share and it pays no dividends. Assuming a constant interest rate of $8%$  per year compounded quarterly, what is the stock's theoretical forward price for delivery in $9$ months?

The discount factor is
$$
d(0,9) = \frac{1}{(1+r/4)^3} = \frac{1}{1.02^3}
$$
Therefore, the forward price is 
$$
F = \frac{S_0}{d(0,9)} = 400(1.02)^3 = 424.4832
$$


In [6]:
p = 400
r = 0.08
period = 4
time = 0.75 # in years

rk.compound_interest(p, r, time, periods_per_year=period)

424.48320000000007

### Question 6 (Bounds using different lending and borrowing rate )

Suppose the borrowing rate $r_B = 10%$ compounded annually. However, the lending rate (or equivalently, the interest rate on deposits) is only $8%$ compounded annually. 

Compute the difference between the upper and lower bounds on the price of an perpetuity that pays A = `$10,000` per year.

 The upper bound on the price of the annuity is
$$
\frac{r_L}{A} = \frac{0.08}{10000} = 125,000
$$
 The lower bound on the price of the annuity is 
$$
\frac{A}{r_B} = \frac{10000}{0.1} = 100,000
$$
 The difference is 25,000.

In [7]:
A = 10000
rb = 0.1
rl = 0.08
perpetuity_b = A / rb
perpetuity_l = A / rl
perpetuity_l - perpetuity_b

25000.0

### Question 7 (Value of a Forward contract at an intermediate time)

Suppose we hold a forward contract on a stock with expiration $6$ months from now. We entered into this contract $6$ months ago so that when we entered into the contract, the expiration was $T = 1$ year. The stock price $6$ months ago was $S_0 = 100$, the current stock price is $125$ and the current interest rate is $r = 10%$ compounded semi-annually. (This is the same rate that prevailed $6$ months ago.) 

What is the current value of our forward contract?

The forward price $F_0$ at the time we entered into the forward contract is given by 
$$
F_0 = S_0/d(0,T) = S_0(1+r/2)^2 = 110.25
$$
The forward price $F_t$ at the current time t for a forward contract with expiration 6 months is given by 
$$
F_t = S_t/d(t,T) = S_t(1+r/2) = 131.25
$$
Therefore, the value is 
$$
f_t = (F_t-F_0)d(t,T) = (131.25-110.25)/(1+r/2) = 20
$$


In [8]:
S0 = 100
St = 125
rate = 0.1
periods = 2

F0 = S0 / rk.discount(2, rate / periods)
Ft = St / rk.discount(1, rate / periods)
ft = (Ft - F0) * rk.discount(1, rate / periods)

round(ft, 1)

20.0